In [3]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import chi2

In [4]:
# Datos iniciales
S_sc = 500  # MVA
alpha = 3
Beta = 3
kVll1 = 12.47  # kV

# Matrices de impedancias
L12 = 2000 * 0.000189  # MILLAS
L34 = 2500 * 0.000189  # MILLAS
zt = 0.01 + 1j * 0.06
ZT = np.diag([zt, zt, zt])

ZbaseH = (12.47**2) / 6  # ohms
ZbaseL = (4.16**2) / 6  # ohms
Z12 = np.array([[0.4013 + 1j * 1.4133, 0.0953 + 1j * 0.8515, 0.0953 + 1j * 0.7266],
                [0.0953 + 1j * 0.8515, 0.4013 + 1j * 1.4133, 0.0953 + 1j * 0.7802],
                [0.0953 + 1j * 0.7266, 0.0953 + 1j * 0.7802, 0.4013 + 1j * 1.4133]])
Z34 = Z12
Z12_Total = Z12 * L12 / ZbaseH
Z34_Total = Z34 * L34 / ZbaseL

# Matriz de admitancia Ybus
Y_12 = np.linalg.inv(Z12_Total)
Y_34 = np.linalg.inv(Z34_Total)
Y_T = np.linalg.inv(ZT)
O = np.zeros((3, 3))
Ybus = np.block([
    [Y_12, -Y_12, O, O],
    [-Y_12, Y_12 + Y_T, -Y_T, O],
    [O, -Y_T, Y_T + Y_34, -Y_34],
    [O, O, -Y_34, Y_34]
])

print("Ybus Matrix:")
print(Ybus)

Ybus Matrix:
[[ 31.38883314-66.68741151j -18.53180593+26.42816518j
   -8.35730289+18.18956186j -31.38883314+66.68741151j
   18.53180593-26.42816518j   8.35730289-18.18956186j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j        ]
 [-18.53180593+26.42816518j  34.55449995-68.77881996j
  -12.88338041+21.80385813j  18.53180593-26.42816518j
  -34.55449995+68.77881996j  12.88338041-21.80385813j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j        ]
 [ -8.35730289+18.18956186j -12.88338041+21.80385813j
   26.80551667-63.72185388j   8.35730289-18.18956186j
   12.88338041-21.80385813j -26.80551667+63.72185388j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j        ]
 [-31.38883314+66.68741151j  18.53180593-26.42816518j
    8.

In [25]:
# Datos adicionales y condiciones iniciales
Z = np.array([1.01, 0.99, 1, 0.97, 0.98, 0.99, 0.98, 0.98, 0.99, 0.90, 0.91, 0.99,
              0.68, 1.01, 1.23, 0, 0, 0, 0, 0, 0, 0.68, 1.01, 1.23,
              1.5400/2, 2.0192/2, 2.4660/2, 0.2188/2, -0.1552/2, 0.0102/2])

PD = np.array([1275/2000, 1800/2000, 2375/2000])  # per unit
SD = PD / np.array([0.85, 0.90, 0.95])  # per unit
QD = np.sqrt(SD**2 - PD**2)  # per unit
V1 = np.array([1, 1, 1])  # slack bus voltages
theta1 = np.array([0, -120, 120]) * np.pi / 180  # slack bus angles in radians

w1 = 100 * np.ones(15)
w2 = (10**5) * np.ones(6)
w3 = 100 * np.ones(9)
w = np.concatenate((w1, w2, w3))

x0 = np.array([0.998972937, 0.988937218, 0.998022426, 0.993959919, 0.982138102, 0.9958213,
               0.981127621, 0.979281974, 0.987178896, 0.926054048, 0.917011078, 0.993255017,
               -0.002372836, -2.105070902, 2.079901885, -0.044086962, -2.167984452, 2.004893842,
               -0.067790489, -2.305505476, 1.838524271])

# Función objetivo
def objective(x):
    f = 0
    V= [x[i] * np.exp(1j * theta1[i]) for i in range(3)] + [0]*9  # Voltajes en el slack bus con sus ángulos
    for i in range(9):  # Voltajes en los nodos restantes con sus ángulos respectivos
        V[3 + i] = x[3 + i] * np.exp(1j * x[12 + i])

    I = Ybus @ V  # Corrientes calculadas a partir de los voltajes y Ybus
    
    # Crear h
    h = np.zeros(30)
    h[:12] = x[:12]  # Las magnitudes de los voltajes
    h[12:24] = np.abs(I)  # Las magnitudes de las corrientes
    h[24:27] = np.array([(V[i]* np.conj(I[i])).real for i in range(3)])  # Potencia activa
    h[27:30] = np.array([(V[i]* np.conj(I[i])).imag for i in range(3)])  # Potencia reactiva
   
    for i in range(30):
        f += w[i]*(Z[i]-h[i])**2
    return f





In [26]:
# Optimización
result = minimize(objective, x0, method='TNC', options={'maxiter': 10000, 'maxfun': 50000, 'disp': True})

# Si la optimización fue exitosa
if result.success:
    x = result.x
    print("Optimization succeeded using TNC.")

    # Grados de libertad y chequeo de datos si la optimización fue exitosa
    m = 30  # Ajusta esto según la cantidad de mediciones que tienes
    n = len(x)
    DOF = m - n
    FVAL = objective(x)
    p_value = 1 - chi2.cdf(FVAL, DOF)
    conf = 0.95
    J = chi2.ppf(1-conf, DOF)

    print(f"Degrees of Freedom: {DOF}")
    print(f"Chi-squared Test Statistic: {FVAL}")
    print(f"J value (threshold): {J}")
    print(f"p-value: {p_value}")
    print(f"Are measurements reliable? {'Yes' if J > FVAL else 'No'}")
else:
    print("Optimization failed.")
    print("Error details:", result.message)

/var/folders/4v/j28kxpb15rzgkf7qhpx8gg640000gn/T/ipykernel_37727/3161458171.py:2: OptimizeWarning: Unknown solver options: maxiter
  result = minimize(objective, x0, method='TNC', options={'maxiter': 10000, 'maxfun': 50000, 'disp': True})
  NIT   NF   F                       GTG
    0    1  4.539676297408459E+01   7.62027979E+10
tnc: fscale = 2.34863e-06
    1    7  2.535001878792864E+01   3.72612130E+09
    2   20  1.873595746800230E+01   1.15592053E+10
    3   32  7.816754331263601E+00   1.26063377E+10
    4   50  7.626056786876474E+00   9.71028023E+09
    5   56  4.011495801400650E+00   2.96072262E+09
    6   68  2.448500885502235E+00   3.32835500E+09
    7   81  1.823089340979407E+00   1.63907008E+09
    8   97  1.761320072673118E+00   1.33809818E+09
    9  101  1.364823533273859E+00   5.93227098E+08
   10  107  1.112164005206696E+00   3.14782897E+08
   11  123  1.110822328008581E+00   3.26199656E+08
   12  126  1.093935122605723E+00   1.76620400E+08
   13  142  1.084609242710875E+

Optimization succeeded using TNC.
Degrees of Freedom: 9
Chi-squared Test Statistic: 0.8715884925309899
J value (threshold): 3.325112843066816
p-value: 0.9996808272650544
Are measurements reliable? Yes


  558 4788  8.719803129431472E-01   1.78462440E+05
  559 4793  8.719137251379315E-01   8.19599925E+05
  560 4807  8.718496946105948E-01   9.45104337E+05
  561 4811  8.718329295105011E-01   9.21773542E+05
  562 4826  8.717923832878552E-01   6.73557477E+05
  563 4830  8.717772744034190E-01   5.02395365E+05
  564 4836  8.717084815995962E-01   1.23553584E+05
  565 4852  8.717074160291504E-01   1.41525940E+05
  566 4868  8.716973232793834E-01   2.15640116E+05
  567 4872  8.716448272687419E-01   2.48924238E+05
  568 4878  8.716387894046931E-01   2.92542356E+05
  569 4897  8.716026177753193E-01   1.06109815E+05
  570 4914  8.715946047873451E-01   8.60230952E+04
  571 4917  8.715922540486135E-01   7.31993405E+04
  572 4924  8.715893611089967E-01   7.31750623E+04
tnc: |xn-xn-1] = 1.38704e-08 -> convergence
  573 4927  8.715884925309899E-01   6.79399845E+04
tnc: Converged (|x_n-x_(n-1)| ~= 0)


In [27]:
radian_to_degree = 180 / np.pi

# Imprimir las soluciones
print("Voltajes calculados en cada nodo (pu):")
print(f"mV1a = {x[0]} pu")
print(f"mV1b = {x[1]} pu")
print(f"mV1c = {x[2]} pu")
print(f"mV2a = {x[3]} pu")
print(f"mV2b = {x[4]} pu")
print(f"mV2c = {x[5]} pu")
print(f"mV3a = {x[6]} pu")
print(f"mV3b = {x[7]} pu")
print(f"mV3c = {x[8]} pu")
print(f"mV4a = {x[9]} pu")
print(f"mV4b = {x[10]} pu")
print(f"mV4c = {x[11]} pu")

print("\nÁngulos en los nodos (grados):")
print(f"aV2a = {x[12] * radian_to_degree} deg")
print(f"aV2b = {x[13] * radian_to_degree} deg")
print(f"aV2c = {x[14] * radian_to_degree} deg")
print(f"aV3a = {x[15] * radian_to_degree} deg")
print(f"aV3b = {x[16] * radian_to_degree} deg")
print(f"aV3c = {x[17] * radian_to_degree} deg")
print(f"aV4a = {x[18] * radian_to_degree} deg")
print(f"aV4b = {x[19] * radian_to_degree} deg")
print(f"aV4c = {x[20] * radian_to_degree} deg")

Voltajes calculados en cada nodo (pu):
mV1a = 0.9989969703096981 pu
mV1b = 0.9888143676077572 pu
mV1c = 0.9980126388630692 pu
mV2a = 0.993948904188911 pu
mV2b = 0.9820263507016888 pu
mV2c = 0.9959786150537697 pu
mV3a = 0.9811709591655112 pu
mV3b = 0.9795473046487148 pu
mV3c = 0.9873795966391625 pu
mV4a = 0.9258692472271923 pu
mV4b = 0.9200650702472758 pu
mV4c = 0.9912482007885564 pu

Ángulos en los nodos (grados):
aV2a = -0.11927703218290955 deg
aV2b = -120.61876982878799 deg
aV2c = 119.17457044211788 deg
aV3a = -2.4949281436338073 deg
aV3b = -124.21117933309253 deg
aV3c = 114.89768001599717 deg
aV4a = -4.083798583770456 deg
aV4b = -131.98485499359987 deg
aV4c = 105.45327399509381 deg
